In [1]:
#uses DotProduct_Systolic_#N - v1.1x
#uses DotProduct_#N_noHR_C#mamcs_A#baseAdderSize - v3.7
#uses DotProduct_#N_noHR_M_A#baseAdderSize - v3.7
#uses RegisteredMultiplier  - v1.0x

In [2]:
import sys
import math

In [3]:
#M>=1
M = input('Enter M: ')

Enter M: 5


In [4]:
#N>=1
N = input('Enter N: ')

Enter N: 30


In [5]:
#>=1, <=M
PR = input('Enter No Of Parallel Row (empty for 1): ')

Enter No Of Parallel Row (empty for 1): 2


In [6]:
M = int(M)
if(M<1):
    M=1

N = int(N)
if(N<1):
    N=1

if not PR:
    PR=1
else:
    PR = int(PR)
    if PR>M:
        PR = M

In [7]:
#MAMCS>=1
MAMCS = input('Enter Multiply-Add Max Component Size (empty for {}): '.format(N))

Enter Multiply-Add Max Component Size (empty for 30): 12


In [8]:
if not MAMCS:
    MAMCS = N
else:
    MAMCS = int(MAMCS)
    if MAMCS>N:
        MAMCS = N;
    elif(MAMCS<1):
        MAMCS=1

In [9]:
#AdderSize>=2
if MAMCS<N:
    AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')

Enter maximum size allowed for base adders (empty for 2): 3


In [10]:
RS=math.ceil(M/PR)

In [11]:
if RS>1:
    Enable_Vector_Latch = input('Enable Vector Latch? (empty or 0 to disable, 1 to enable): ')

Enable Vector Latch? (empty or 0 to disable, 1 to enable): 1


In [12]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10): ')

Enter IN_WIDTH default value (empty for 10): 


In [13]:
#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH default value (empty for 1): ')

Enter INPUT_REG_DEPTH default value (empty for 1): 


In [14]:
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH default value (empty for 1): ')

Enter MULT_PIPE_DEPTH default value (empty for 1): 


In [15]:
NumOfComponents = math.ceil(N/MAMCS)
if MAMCS==N:
    AdderSize=0
else:
    if not AdderSize:
        AdderSize=2
    else:
        AdderSize = int(AdderSize)
        if AdderSize<2:
            AdderSize=2
        if AdderSize>NumOfComponents:
            AdderSize=NumOfComponents

if RS==1:
    Enable_Vector_Latch = 0;
else:
    if not Enable_Vector_Latch:
        Enable_Vector_Latch = 0
    else:
        Enable_Vector_Latch = int(Enable_Vector_Latch)

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if(IN_WIDTH<1):
        IN_WIDTH=1

if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0

In [16]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    if Enable_Vector_Latch>0:
        of=open("./MatrixVectorMultiply_{}_{}_{}PR_noHR_C{}_A{}_VL.v".format(M, N, PR, MAMCS, AdderSize), 'w+')
    else:
        of=open("./MatrixVectorMultiply_{}_{}_{}PR_noHR_C{}_A{}_NVL.v".format(M, N, PR, MAMCS, AdderSize), 'w+')

In [17]:
print("`timescale 1ns / 1ps\n", file=of)
if Enable_Vector_Latch>0:
    print("module MatrixVectorMultiply_{}_{}_{}PR_noHR_C{}_A{}_VL".format(M, N, PR, MAMCS, AdderSize), file=of)
else:
    print("module MatrixVectorMultiply_{}_{}_{}PR_noHR_C{}_A{}_NVL".format(M, N, PR, MAMCS, AdderSize), file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
lgRS=math.ceil(math.log2(RS))
if lgRS<=1:
    print("output reg rowSetInNo = 0,", file=of)
else:
    print("output reg [{}:0] rowSetInNo = 0,".format(lgRS-1), file=of)
print("input inReady,", file=of)
for j in range(PR):
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N):
        print("AE{}R{}, ".format(i, j), end='', file=of)
    print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("VE{}, ".format(i), end='', file=of)
print(file=of)
ar=RS*PR-M
ME = PR-ar-1
if ME==0:
    st1="E0"
else:
    st1="E0toE{}".format(ME)
print("output ", end='', file=of)
print(st1, end='', file=of)
print("Ready,", file=of)
if ar==0:
    print("output reg NUCReady = 0, //not used", file=of)
else:
    if ME==(PR-2):
        st2="E{}".format(PR-1)
    else:
        st2="E{}toE{}".format(ME+1, PR-1)
    print("output ", end='', file=of)
    print(st2, end='', file=of)
    print("Ready,", file=of)
if lgRS<=1:
    print("output reg rowSetOutNo = {},".format(RS-1), file=of)
else:
    print("output reg [{}:0] rowSetOutNo = {},".format(lgRS-1, RS-1), file=of)
lgn=math.ceil(math.log2(N))
al=-1+lgn
if al<0:
    print("output signed [(2*IN_WIDTH){}:0] ".format(al), end='',file=of)
elif al==0:
    print("output signed [(2*IN_WIDTH):0] ", end='',file=of)
else:
    print("output signed [(2*IN_WIDTH)+{}:0] ".format(al), end='',file=of)
for i in range(PR):
    print("E{}, ".format(i), end='',file=of)
print(file=of)
print("output early", end='', file=of)
print(st1, end='', file=of)
print("Ready,", file=of)
if ar==0:
    print("output reg NUearlyCReady = 0 //not used", file=of)
else:
    print("output early", end='', file=of)
    print(st2, end='', file=of)
    print("Ready", file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module MatrixVectorMultiply_5_30_2PR_noHR_C12_A3_VL
#(
parameter IN_WIDTH = 10,
parameter INPUT_REG_DEPTH = 1,
parameter MULT_PIPE_DEPTH = 1
)(
input clk, reset, enable,
output reg [1:0] rowSetInNo = 0,
input inReady,
input signed [IN_WIDTH-1:0] AE0R0, AE1R0, AE2R0, AE3R0, AE4R0, AE5R0, AE6R0, AE7R0, AE8R0, AE9R0, AE10R0, AE11R0, AE12R0, AE13R0, AE14R0, AE15R0, AE16R0, AE17R0, AE18R0, AE19R0, AE20R0, AE21R0, AE22R0, AE23R0, AE24R0, AE25R0, AE26R0, AE27R0, AE28R0, AE29R0, 
input signed [IN_WIDTH-1:0] AE0R1, AE1R1, AE2R1, AE3R1, AE4R1, AE5R1, AE6R1, AE7R1, AE8R1, AE9R1, AE10R1, AE11R1, AE12R1, AE13R1, AE14R1, AE15R1, AE16R1, AE17R1, AE18R1, AE19R1, AE20R1, AE21R1, AE22R1, AE23R1, AE24R1, AE25R1, AE26R1, AE27R1, AE28R1, AE29R1, 
input signed [IN_WIDTH-1:0] VE0, VE1, VE2, VE3, VE4, VE5, VE6, VE7, VE8, VE9, VE10, VE11, VE12, VE13, VE14, VE15, VE16, VE17, VE18, VE19, VE20, VE21, VE22, VE23, VE24, VE25, VE26, VE27, VE28, VE29, 
output E0Ready,
output E1Ready,
output reg 

In [18]:
if Enable_Vector_Latch>0:
    print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N-1):
        print("SVE{}, ".format(i), end='', file=of)
    print("SVE{};".format(N-1), file=of)
    print(file=of)

reg signed [IN_WIDTH-1:0] SVE0, SVE1, SVE2, SVE3, SVE4, SVE5, SVE6, SVE7, SVE8, SVE9, SVE10, SVE11, SVE12, SVE13, SVE14, SVE15, SVE16, SVE17, SVE18, SVE19, SVE20, SVE21, SVE22, SVE23, SVE24, SVE25, SVE26, SVE27, SVE28, SVE29;



In [19]:
if(ar>0): #RS*PR > M, RS>=2
    print("wire inReadyC = (rowSetInNo!={}) & inReady;".format(RS-1), file=of)
print(file=of)
for i in range (PR):
    if N==1:
        print("RegisteredMultiplier", file=of)
    elif MAMCS==N:
        print("DotProduct_Systolic_{}".format(N), file=of)
    elif MAMCS==1:
        print("DotProduct_{}_noHR_M_A{}".format(N, AdderSize), file=of)  
    else:
        print("DotProduct_{}_noHR_C{}_A{}".format(N, MAMCS, AdderSize), file=of)
    print("#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )", file=of)
    print("DP_{} (clk, reset, enable,".format(i), file=of)
    if(PR-i<=ar):
        print("inReadyC,", file=of)
    else:
        print("inReady,", file=of)
    for j in range(N):
        print("AE{}R{}, ".format(j, i), end='', file=of)
    print(file=of)
    if Enable_Vector_Latch>0:
        for j in range(N):
            print("SVE{}, ".format(j), end='', file=of)
    else:
        for j in range(N):
            print("VE{}, ".format(j), end='', file=of)        
    print(file=of)
    if i==0:
        print(st1+"Ready,", file=of)
    elif i==PR-ar:
        print(st2+"Ready,", file=of)
    else:
        print("DPOutReady{}, //not used".format(i), file=of)
    print("E{},".format(i), file=of)
    if i==0:
        print("early"+st1+"Ready", file=of)
    elif i==PR-ar:
        print("early"+st2+"Ready", file=of)
    else:
        print("DPearlyOutReady{} //not used".format(i), file=of)
    print(");\n", file=of)

wire inReadyC = (rowSetInNo!=2) & inReady;

DotProduct_30_noHR_C12_A3
#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
DP_0 (clk, reset, enable,
inReady,
AE0R0, AE1R0, AE2R0, AE3R0, AE4R0, AE5R0, AE6R0, AE7R0, AE8R0, AE9R0, AE10R0, AE11R0, AE12R0, AE13R0, AE14R0, AE15R0, AE16R0, AE17R0, AE18R0, AE19R0, AE20R0, AE21R0, AE22R0, AE23R0, AE24R0, AE25R0, AE26R0, AE27R0, AE28R0, AE29R0, 
SVE0, SVE1, SVE2, SVE3, SVE4, SVE5, SVE6, SVE7, SVE8, SVE9, SVE10, SVE11, SVE12, SVE13, SVE14, SVE15, SVE16, SVE17, SVE18, SVE19, SVE20, SVE21, SVE22, SVE23, SVE24, SVE25, SVE26, SVE27, SVE28, SVE29, 
E0Ready,
E0,
earlyE0Ready
);

DotProduct_30_noHR_C12_A3
#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
DP_1 (clk, reset, enable,
inReadyC,
AE0R1, AE1R1, AE2R1, AE3R1, AE4R1, AE5R1, AE6R1, AE7R1, AE8R1, AE9R1, AE10R1, AE11R1, AE12R1, AE13R1, AE14R1, AE15R1, AE16R1, AE17R1, AE18R1, AE19R1, AE20R1, AE21R1, AE22R1, AE23R1,

In [20]:
if(RS>1):
    if (Enable_Vector_Latch>0):
        print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
        for i in range(N-1):
            print("LVE{}, ".format(i), end='', file=of)
        print("LVE{};".format(N-1), file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\trowSetInNo <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(inReady) begin", file=of)
    print("\t\t\tif(rowSetInNo=={}) begin".format(RS-1), file=of)
    print("\t\t\t\trowSetInNo <= 0;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\telse begin", file=of)
    if RS==2:
        print("\t\t\t\trowSetInNo <= 1;", file=of)    
    else:
        print("\t\t\t\trowSetInNo <= rowSetInNo+1;", file=of)
    print("\t\t\tend", file=of)
    if Enable_Vector_Latch>0:
        print("\t\t\tif(rowSetInNo==0) begin", file=of)
        print("\t\t\t\t", end='', file=of)
        for j in range(N):
            print("LVE{} <= VE{}; ".format(j, j), end='', file=of)
        print(file=of)
        print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

reg signed [IN_WIDTH-1:0] LVE0, LVE1, LVE2, LVE3, LVE4, LVE5, LVE6, LVE7, LVE8, LVE9, LVE10, LVE11, LVE12, LVE13, LVE14, LVE15, LVE16, LVE17, LVE18, LVE19, LVE20, LVE21, LVE22, LVE23, LVE24, LVE25, LVE26, LVE27, LVE28, LVE29;
always @(posedge clk) begin
	if(reset) begin
		rowSetInNo <= 0;
	end
	else if(enable) begin
		if(inReady) begin
			if(rowSetInNo==2) begin
				rowSetInNo <= 0;
			end
			else begin
				rowSetInNo <= rowSetInNo+1;
			end
			if(rowSetInNo==0) begin
				LVE0 <= VE0; LVE1 <= VE1; LVE2 <= VE2; LVE3 <= VE3; LVE4 <= VE4; LVE5 <= VE5; LVE6 <= VE6; LVE7 <= VE7; LVE8 <= VE8; LVE9 <= VE9; LVE10 <= VE10; LVE11 <= VE11; LVE12 <= VE12; LVE13 <= VE13; LVE14 <= VE14; LVE15 <= VE15; LVE16 <= VE16; LVE17 <= VE17; LVE18 <= VE18; LVE19 <= VE19; LVE20 <= VE20; LVE21 <= VE21; LVE22 <= VE22; LVE23 <= VE23; LVE24 <= VE24; LVE25 <= VE25; LVE26 <= VE26; LVE27 <= VE27; LVE28 <= VE28; LVE29 <= VE29; 
			end
		end
	end
end



In [21]:
if Enable_Vector_Latch>0:
    print("always @(*) begin", file=of)
    print("\tif(rowSetInNo==0) begin", file=of)
    print("\t\t", end='', file=of)
    for j in range(N):
        print("SVE{} = VE{}; ".format(j,j), end='', file=of)
    print(file=of)
    print("\tend", file=of)
    print("\telse begin", file=of)
    print("\t\t", end='', file=of)
    for j in range(N):
        print("SVE{} = LVE{}; ".format(j, j), end='', file=of)
    print(file=of)
    print("\tend", file=of)
    print("end\n", file=of)

always @(*) begin
	if(rowSetInNo==0) begin
		SVE0 = VE0; SVE1 = VE1; SVE2 = VE2; SVE3 = VE3; SVE4 = VE4; SVE5 = VE5; SVE6 = VE6; SVE7 = VE7; SVE8 = VE8; SVE9 = VE9; SVE10 = VE10; SVE11 = VE11; SVE12 = VE12; SVE13 = VE13; SVE14 = VE14; SVE15 = VE15; SVE16 = VE16; SVE17 = VE17; SVE18 = VE18; SVE19 = VE19; SVE20 = VE20; SVE21 = VE21; SVE22 = VE22; SVE23 = VE23; SVE24 = VE24; SVE25 = VE25; SVE26 = VE26; SVE27 = VE27; SVE28 = VE28; SVE29 = VE29; 
	end
	else begin
		SVE0 = LVE0; SVE1 = LVE1; SVE2 = LVE2; SVE3 = LVE3; SVE4 = LVE4; SVE5 = LVE5; SVE6 = LVE6; SVE7 = LVE7; SVE8 = LVE8; SVE9 = LVE9; SVE10 = LVE10; SVE11 = LVE11; SVE12 = LVE12; SVE13 = LVE13; SVE14 = LVE14; SVE15 = LVE15; SVE16 = LVE16; SVE17 = LVE17; SVE18 = LVE18; SVE19 = LVE19; SVE20 = LVE20; SVE21 = LVE21; SVE22 = LVE22; SVE23 = LVE23; SVE24 = LVE24; SVE25 = LVE25; SVE26 = LVE26; SVE27 = LVE27; SVE28 = LVE28; SVE29 = LVE29; 
	end
end



In [22]:
if(RS>1):
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\trowSetOutNo <= {};".format(RS-1), file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(early"+st1+"Ready) begin", file=of)
    print("\t\t\tif(rowSetOutNo=={}) begin".format(RS-1), file=of)
    print("\t\t\t\trowSetOutNo <= 0;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\telse begin", file=of)
    if RS==2:
        print("\t\t\t\trowSetOutNo <= 1;", file=of)
    else:
        print("\t\t\t\trowSetOutNo <= rowSetOutNo+1;", file=of)
    print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

always @(posedge clk) begin
	if(reset) begin
		rowSetOutNo <= 2;
	end
	else if(enable) begin
		if(earlyE0Ready) begin
			if(rowSetOutNo==2) begin
				rowSetOutNo <= 0;
			end
			else begin
				rowSetOutNo <= rowSetOutNo+1;
			end
		end
	end
end



In [23]:
print("endmodule", file=of)

endmodule


In [24]:
if __Print_To_File>0:
    of.close()